# Volumetric heat maps

In [ ]:
samples = 10

In [ ]:
import time
import random
import math
import numpy as np

In [ ]:
%env QRACK_QUNITMULTI_DEVICES 1
from pyqrack import QrackSimulator, QrackCircuit

In [ ]:
def mcx(circ, c, q):
    circ.ucmtrx([c], [0, 1, 1, 0], q, 1)

def mcy(circ, c, q):
    circ.ucmtrx([c], [0, -1j, 1j, 0], q, 1)

def mcz(circ, c, q):
    circ.ucmtrx([c], [1, 0, 0, -1], q, 1)

def macx(circ, c, q):
    circ.ucmtrx([c], [0, 1, 1, 0], q, 0)

def macy(circ, c, q):
    circ.ucmtrx([c], [0, -1j, 1j, 0], q, 0)

def macz(circ, c, q):
    circ.ucmtrx([c], [1, 0, 0, -1], q, 0)

def rand_u3(circ, q):
    th = random.uniform(0, 4 * math.pi)
    ph = random.uniform(0, 4 * math.pi)
    lm = random.uniform(0, 4 * math.pi)

    c = math.cos(th / 2)
    s = math.sin(th / 2)

    op = []
    op.append(c)
    op.append(-np.exp(1j * lm) * s)
    op.append(np.exp(1j * ph) * s)
    op.append(np.exp(1j * (ph + lm)) * c)

    circ.mtrx(op, q)

def bench(width, depth, sdrp):
    sim = QrackSimulator(width)
    sim.set_sdrp(sdrp)
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    two_qubit_gates = mcx, mcy, mcz, macx, macy, macz
    
    colLen = math.floor(math.sqrt(width))
    while ((math.floor(width / colLen) * colLen) != width):
        colLen = colLen - 1
    rowLen = width // colLen;

    time_results = []
    fidelity_results = []

    for _ in range(samples):
        circ = QrackCircuit()
        d_time_results = []
        d_fidelity_results = []
        
        for i in range(depth):
            # Single bit gates
            for j in range(width):
                rand_u3(circ, j)

            gate = gateSequence[0]
            gateSequence.pop(0)
            gateSequence.append(gate)

            for row in range(1, rowLen, 2):
                for col in range(colLen):
                    tempRow = row
                    tempCol = col

                    tempRow = tempRow + (1 if (gate & 2) else -1);
                    if colLen != 1:
                        tempCol = tempCol + (1 if (gate & 1) else 0)

                    if (tempRow < 0) or (tempCol < 0) or (tempRow >= rowLen) or (tempCol >= colLen):
                        continue
                    
                    b1 = row * colLen + col
                    b2 = tempRow * colLen + tempCol

                    # Two bit gates
                    choice = random.choice(two_qubit_gates)
                    choice(circ, b1, b2)

            sim.reset_all()
            start = time.time()
            circ.run(sim)
            d_time_results.append(time.time() - start)
            d_fidelity_results.append(sim.get_unitary_fidelity())

        if not time_results:
            for i in range(len(d_time_results)):
                time_results.append(d_time_results[i] / samples)
            for i in range(len(d_fidelity_results)):
                fidelity_results.append(d_fidelity_results[i] / samples)
        else:
            for i in range(len(d_time_results)):
                time_results[i] += d_time_results[i] / samples
            for i in range(len(d_fidelity_results)):
                fidelity_results[i] += d_fidelity_results[i] / samples
        
    return (time_results, fidelity_results)

In [ ]:
qrack_time_results = {}
qrack_fidelity_results = {}

for n in [36, 49, 64, 81, 100]:
    qrack_time_results[n] = []
    qrack_fidelity_results[n] = []

    sdrp = 1

    while (sdrp > 0.01):
        width_results = []
        
        # Run the benchmarks
        tmp = bench(n, n, sdrp)
        
        qrack_time_results[n].append(tmp[0])
        qrack_fidelity_results[n].append(tmp[1])
        
        sdrp -= 0.0125

print(qrack_time_results, qrack_fidelity_results)

## Results

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 36
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_time_results[36]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=36.455696203)

fig.savefig('heat_map_time_36.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 49
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_time_results[49]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=49.620253165)

fig.savefig('heat_map_time_49.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 64
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_time_results[64]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=64.810126582)

fig.savefig('heat_map_time_64.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 81
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_time_results[81]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=82.025316456)

fig.savefig('heat_map_time_81.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 100
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_time_results[100]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=101.2658)

fig.savefig('heat_map_time_100.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 36
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_fidelity_results[36]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=36.455696203)

fig.savefig('heat_map_fidelity_36.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 49
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_fidelity_results[49]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=49.620253165)

fig.savefig('heat_map_fidelity_49.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 64
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_fidelity_results[64]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=64.810126582)

fig.savefig('heat_map_fidelity_64.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 81
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_fidelity_results[81]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=82.025316456)

fig.savefig('heat_map_fidelity_81.png', dpi=100)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (9, 9))
# plt.title("Fidelity vs. layer depth, 54 qubits, A100 GPU")
plt.xlabel("Circuit layers")
plt.ylabel("SDRP")

left = 1
right = 100
bottom = 0.0125
top = 1
extent = [left, right, bottom, top]

h = qrack_fidelity_results[100]

plt.imshow(h, cmap='hot', interpolation='nearest', extent=extent, aspect=101.2658)

fig.savefig('heat_map_fidelity_100.png', dpi=100)

In [ ]:
import platform

print(platform.machine())
print(platform.version())
print(platform.platform())
print(platform.uname())
print(platform.system())
print(platform.processor())

In [ ]:
import subprocess

line_as_bytes = subprocess.check_output("nvidia-smi -L", shell=True)
line = line_as_bytes.decode("ascii")
_, line = line.split(":", 1)
line, _ = line.split("(")
print(line.strip())

In [ ]:
import cpuinfo
cpuinfo.get_cpu_info()['brand_raw']